# Olist product categories bad reviews analysis

# Import

In [3]:
import pandas as pd

from olist.review import Review
from olist.data import Olist

## Data collection

In [4]:
# Get dataset with product category name
product_category = Review().get_training_data()

# Get dataset with reviews and estimated delivery date
data = Olist().get_data()
reviews_data = data['order_reviews']
orders_data = data['orders']

# Build a dataset merging the 3 datasets
df = product_category.merge(reviews_data)
df = df.merge(orders_data)

## Data cleaning

In [6]:
from olist.utils import clean

# Remove reviews for orders delivered before expected
df = df[(df['order_delivered_customer_date'] <= df['order_estimated_delivery_date'])]

# Remove undelivered reviews
df = df[df['order_status'] == "delivered"]

# Keep only text columns and review score
df = df[['order_id', 'product_category_name', 'review_comment_title', 'review_comment_message', 'review_score']]

# combine review title and review message
df = df.dropna(subset=['review_comment_title', 'review_comment_message'])

df['title_comment'] = df["review_comment_title"].fillna('') + " " \
    + df['review_comment_message'].fillna('')

# Clean reviews text
df['clean_text'] = df['title_comment'].apply(clean)

df.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
9,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo,aparelho eficiente. no site a marca do aparelh...,4,recomendo aparelho eficiente. no site a marca ...,recomendo aparelho eficiente site marca aparel...
15,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",5,"Super recomendo Vendedor confiável, produto ok...",super recomendo vendedor confiavel produto ok ...
22,4fc44d78867142c627497b60a7e0228a,beleza_saude,Ótimo,Loja nota 10,5,Ótimo Loja nota 10,otimo loja nota
36,37e7875cdce5a9e5b3a692971f370151,esporte_lazer,Muito bom.,Recebi exatamente o que esperava. As demais en...,4,Muito bom. Recebi exatamente o que esperava. A...,bom recebi exatamente esperava demais encomend...
38,e029f708df3cc108b3264558771605c6,pet_shop,Bom,"Recomendo ,",5,"Bom Recomendo ,",bom recomendo


## Bad reviews analysis per category

In [7]:
# Groupby product category and aggregate mean, min, max review scores
product_performance = df.groupby('product_category_name').agg({'review_score': ['count', 'mean', 'min', 'max']}).sort_values([('review_score', 'mean')], ascending=False)

# Keep categories that have more than 100 reviews
product_performance = product_performance[product_performance[('review_score', 'count')] >= 100]

product_performance.tail(10)

review_score                  
                                         count      mean min max
product_category_name                                           
automotivo                                 441  4.049887   1   5
construcao_ferramentas_construcao          134  4.022388   1   5
eletrodomesticos                           118  3.966102   1   5
moveis_decoracao                           454  3.925110   1   5
cama_mesa_banho                            866  3.866051   1   5
informatica_acessorios                     554  3.857401   1   5
telefonia                                  354  3.850282   1   5
relogios_presentes                         802  3.847880   1   5
bebes                                      246  3.776423   1   5
moveis_escritorio                          119  3.386555   1   5

In [9]:
# Filter out the relogios_presentes category
relogios_presentes = df[df['product_category_name'].isin(['relogios_presentes'])]

# Filter out bad reviews
relogios_presentes_bad_reviews = relogios_presentes[relogios_presentes['review_score'].isin([1])]

relogios_presentes_bad_reviews.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
338,43050a51a257b17d51295e07650cb654,relogios_presentes,Lamenetável...,"Boa Noite\r\n\r\né lamentável, esta loja que t...",1,"Lamenetável... Boa Noite\r\n\r\né lamentável, ...",lamenetavel boa noite lamentavel loja tantas c...
2859,836f9b5adb4fe06d8980fb71fa06b691,relogios_presentes,PÉSSIMO SERVIÇO,Já solicitei a troca porém até o momento nada....,1,PÉSSIMO SERVIÇO Já solicitei a troca porém at...,pessimo servico ja solicitei troca porem ate m...
3248,ae820ecd285ee7d0db7eac6f69049b36,relogios_presentes,Péssima loja,"Comprei um relogio, veio outro, mandei msgs, n...",1,"Péssima loja Comprei um relogio, veio outro, m...",pessima loja comprei relogio veio outro mandei...
3817,232ac1c81c6a84fd76f9653b248b5d92,relogios_presentes,Produto falsificado,O produto é falsificado. Não veio com manual e...,1,Produto falsificado O produto é falsificado. N...,produto falsificado produto falsificado nao ve...
4268,da0c01c648efe58143eae5e7c1a38ac0,relogios_presentes,NÃO entregue,Comprei 2 relógios mas só chegou 1. Já mandei ...,1,NÃO entregue Comprei 2 relógios mas só chegou ...,nao entregue comprei relogios so chegou ja man...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode bad reviews text
# Tuned TFidfvectorizer
vec = TfidfVectorizer(ngram_range=(2, 2), min_df=0.01, max_df=0.05).fit(relogios_presentes_bad_reviews.clean_text)

# Transform text to vectors
vectors = vec.transform(relogios_presentes_bad_reviews.clean_text)

# Sum of tfidf weighting by word
sum_tfidf = vectors.sum(axis=0)

# Get the word and associated weight
tfidf_list = [(word, sum_tfidf[0, idx]) for word, idx in vec.vocabulary_.items()]

# Sort
sorted_tfidf_list = sorted(tfidf_list, key=lambda x: x[1], reverse=True)

# Display the sorted list
sorted_tfidf_list

[('produto entregue', 4.823299861915626),
 ('ainda nao', 3.8500301309682508),
 ('propaganda enganosa', 3.6974097851505032),
 ('produto defeito', 3.4523848836568005),
 ('nao original', 3.2173656380879407),
 ('relogio veio', 3.2145017859559015),
 ('recebi relogio', 2.977055535926564),
 ('nao funciona', 2.8236647422272614),
 ('outro modelo', 2.696081824765045),
 ('entregue nao', 2.639529201040295),
 ('nao gostei', 2.632932643390745),
 ('relogio nao', 2.6108160304768457),
 ('relogio falso', 2.5386933138724315),
 ('nao veio', 2.435464016235141),
 ('errado nao', 2.418869502203795),
 ('troca produto', 2.411741318588243),
 ('produto falsificado', 2.3316511085499565),
 ('nao vale', 2.2588458012151515),
 ('veio defeito', 2.2537904159474897),
 ('diferente comprei', 2.005157255413781),
 ('insatisfacao negativo', 2.0),
 ('produto falso', 2.0),
 ('nao consigo', 1.9982255344127589),
 ('solicitei devolucao', 1.9838122896876063),
 ('quero devolver', 1.8984986276602025),
 ('agora nao', 1.892892708680781

## Tracking sellers of counterfeit products

In [11]:
# Get seller ID
sellers = data['order_items'].merge(relogios_presentes_bad_reviews)

# Filter out reviews with words associated with conterfeit watches
bad_sellers = sellers[sellers['clean_text'].str.contains("nao original|falso|outro modelo|produto falsificado|produto diferente")]

# Groupby seller id
bad_sellers.groupby('seller_id').agg({'seller_id': ['count']})

# Filter out the one seller with 11 counterfeit related reviews
bad_sellers[bad_sellers["seller_id"] == "2eb70248d66e0e3ef83659f71b244378"].sort_values(by='shipping_limit_date')

bad_sellers

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
4,0987363c91de454d42d084b84a691f99,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 15:06:12,155.97,8.35,relogios_presentes,Relógio falso,"Relógio falso, me admiro com uma loja grande c...",1,"Relógio falso Relógio falso, me admiro com uma...",relogio falso relogio falso admiro loja grande...
7,0c23c36327d99027c61ca570921b0fb7,1,a92930c327948861c015c919a0bcb4a8,6560211a19b47992c3666cc44a7e94c0,2018-07-26 03:50:17,78.00,18.65,relogios_presentes,Réplica,"Infelizmente o relógio não é original, achei u...",1,"Réplica Infelizmente o relógio não é original,...",replica infelizmente relogio nao original ache...
13,232ac1c81c6a84fd76f9653b248b5d92,1,6bca83dbf6e081bccb2b23188577ad36,4869f7a5dfa277a7dca6462dcf3b52b2,2018-07-31 17:55:16,288.00,20.12,relogios_presentes,Produto falsificado,O produto é falsificado. Não veio com manual e...,1,Produto falsificado O produto é falsificado. N...,produto falsificado produto falsificado nao ve...
15,2ca33108764fd34547c251d1a4ad73ef,1,c0eeedfd383f32ae9524a15a8898b129,c60b801f2d52c7f7f91de00870882a75,2018-05-08 22:30:42,210.00,16.35,relogios_presentes,Produto errado e falso,Me foi entregue o modelo errado e não e origin...,1,Produto errado e falso Me foi entregue o model...,produto errado falso entregue modelo errado na...
20,31c9a3f10eb368b62e102608adbec724,1,64b4f32393cbf55be791078e7a2adca0,612170e34b97004b3ba37eae81836b4c,2018-07-31 11:41:16,139.90,19.08,relogios_presentes,Produto não original,Produto não informado como paralelo/réplica,1,Produto não original Produto não informado co...,produto nao original produto nao informado par...
26,3ec53ff8dd23b7e8a90cfb4082cf3f85,1,e0d64dcfaa3b6db5c54ca298ae101d05,2eb70248d66e0e3ef83659f71b244378,2018-06-26 21:30:52,124.90,16.84,relogios_presentes,Péssimo atendimento,O produto demorou aproximadamente 50 dias para...,1,Péssimo atendimento O produto demorou aproxima...,pessimo atendimento produto demorou aproximada...
28,3f7e2597afafe07f94a15e7f5da01135,1,2ffdf10e724b958c0f7ea69e97d32f64,4869f7a5dfa277a7dca6462dcf3b52b2,2018-05-08 20:53:31,179.90,19.14,relogios_presentes,Produto diferente,"Olá, a descrição do produto não corresponde co...",1,"Produto diferente Olá, a descrição do produto ...",produto diferente ola descricao produto nao co...
39,4d280f7e02765cbee0a2ce0359dafb2e,1,b84520a57891e7a8ae2c68741dcc7146,6560211a19b47992c3666cc44a7e94c0,2018-07-03 15:29:48,45.00,7.58,relogios_presentes,Relógio veio danificado,Relógio veio danificado e não é original,1,Relógio veio danificado Relógio veio danifica...,relogio veio danificado relogio veio danificad...
65,74fedc6aa056c9359846751fe21a8676,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 09:04:01,154.91,16.18,relogios_presentes,Produto falsificado,Comprei o relógio casio rose Gold e o que cheg...,1,Produto falsificado Comprei o relógio casio r...,produto falsificado comprei relogio casio rose...
80,860e7b15d9daf5ec2b4357fb6cb57f43,1,3225c54bb2785c33f49f50398fcdb88c,c60b801f2d52c7f7f91de00870882a75,2018-06-08 14:54:46,225.00,16.45,relogios_presentes,Produto FALSIFICADO,Produto barato vendido no camelo por 20 reais ...,1,Produto FALSIFICADO Produto barato vendido no ...,produto falsificado produto barato vendido cam...
